In [1]:
import pyspark
import pyspark.sql.functions as F
from pyspark.sql.types import *

In [2]:
from delta import *
builder = pyspark.sql.SparkSession.builder.appName("TaxisLake")\
    .master('local')\
    .config('spark.executor.memory', '8gb') \
    .config("spark.cores.max", "8") \
    .config("spark.driver.maxResultSize", "8g")\
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [15]:
df = spark\
    .read\
    .format('parquet').option('inferSchema', True)\
    .parquet('../datasets/yellow_tripdata_*.parquet')

In [16]:
df.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: integer (nullable = true)
 |-- airport_fee: integer (nullable = true)



In [32]:
df_yellow = df.withColumnRenamed("VendorID","VendorID")\
.withColumn("pickup_datetime",F.from_unixtime(F.unix_timestamp(F.col("tpep_pickup_datetime"))))\
.withColumn("dropoff_datetime",F.from_unixtime(F.unix_timestamp(F.col("tpep_dropoff_datetime"))))\
.withColumnRenamed("store_and_fwd_flag", "store_and_forward",)\
.withColumnRenamed("RatecodeID","RatecodeID")\
.withColumn("PULocationID",F.col("PULocationID").cast(DoubleType()))\
.withColumn("DOLocationID",F.col("DOLocationID").cast(DoubleType()))\
.withColumn("passenger_count",F.col("passenger_count").cast(LongType()))\
.withColumn("trip_distance",F.col("trip_distance").cast(DoubleType()))\
.withColumn("fare_amount",F.col("fare_amount").cast(DoubleType()))\
.withColumn("extra",F.col("extra").cast(DoubleType()))\
.withColumn("mta_tax",F.col("mta_tax").cast(DoubleType()))\
.withColumn("tip_amount",F.col("tip_amount").cast(DoubleType()))\
.withColumn("tolls_amount",F.col("tolls_amount").cast(DoubleType()))\
.withColumn("total_amount", F.col("total_amount").cast(DoubleType()))\
.withColumn("improvement_surcharge", F.col("improvement_surcharge").cast(DoubleType()))\
.withColumn("payment_type",F.col("payment_type"))\
.withColumn("congestion_surcharge",F.col("congestion_surcharge").cast(IntegerType()))\
.withColumn("airport_fee",F.col("airport_fee").cast(IntegerType()))\
.select(["VendorID","pickup_datetime","dropoff_datetime","store_and_forward",
         "RatecodeID","PULocationID","DOLocationID",
         "passenger_count","trip_distance", "fare_amount", "extra", 
         "mta_tax", "tip_amount", "tolls_amount", "total_amount", "improvement_surcharge", "payment_type",
        "congestion_surcharge", "airport_fee"])


In [19]:
df_yellow.count()

1168462174

In [20]:
df_yellow.createOrReplaceTempView ("Taxi")

In [26]:
df = df.withColumn("Year",F.year(F.from_unixtime(F.unix_timestamp(F.col("tpep_pickup_datetime")))))

In [27]:
%%time
df1 = df.groupby('year').agg(F.count('RatecodeID'))

CPU times: total: 0 ns
Wall time: 13.5 ms


In [28]:
%%time
df1.show()

Py4JJavaError: An error occurred while calling o582.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 72 in stage 21.0 failed 1 times, most recent failure: Lost task 72.0 in stage 21.0 (TID 551) (Orion-PC executor driver): org.apache.spark.sql.execution.QueryExecutionException: Parquet column cannot be converted in file file:/D:/Estudos/Projetos/data-lake-taxis-nyc/datasets/yellow_tripdata_2018-10.parquet. Column: [RatecodeID], Expected: bigint, Found: DOUBLE
	at org.apache.spark.sql.errors.QueryExecutionErrors$.unsupportedSchemaColumnConvertError(QueryExecutionErrors.scala:724)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:278)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:116)
	at org.apache.spark.sql.execution.FileSourceScanExec$$anon$1.hasNext(DataSourceScanExec.scala:554)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIt

In [34]:
df.write.format("delta").mode("overwrite").save("storage") 

In [4]:
df = spark.read.format("delta").load("LARGE_DATAFRAME")